In [1]:
%load_ext autoreload
%autoreload 2
import pudl
from pudl import constants as pc
import logging
import sys
from pathlib import Path
import pandas as pd
pd.options.display.max_columns = None

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

In [15]:
pudl_settings = pudl.workspace.setup.get_defaults()
eia923_tables = pc.pudl_tables['eia923']
eia923_years = [2018]
eia860_tables = pc.pudl_tables['eia860']
eia860_years = [2018] # 2019
sandbox = True

In [16]:
ds = pudl.workspace.datastore.Datastore(
    Path(pudl_settings["pudl_in"]),
    sandbox=sandbox)
# Extract EIA forms 923, 860
eia923_raw_dfs = pudl.extract.eia923.Extractor(ds).extract(eia923_years)
eia860_raw_dfs = pudl.extract.eia860.Extractor(ds).extract(eia860_years)

# Transform EIA forms 923, 860
eia860_transformed_dfs = pudl.transform.eia860.transform(
    eia860_raw_dfs, eia860_tables=eia860_tables)
eia923_transformed_dfs = pudl.transform.eia923.transform(
    eia923_raw_dfs, eia923_tables=eia923_tables)

Extracting eia923 spreadsheet data.
Extracting eia860 spreadsheet data.
Columns for boiler_generator_assn are off: should be 4 but got 8
Columns for generator_existing are off: should be 76 but got 77
Columns for generator_proposed are off: should be 55 but got 56
Columns for generator_retired are off: should be 75 but got 76
Columns for ownership are off: should be 14 but got 15
Columns for plant are off: should be 46 but got 48
Columns for utility are off: should be 20 but got 21
Transforming raw EIA 860 DataFrames for ownership_eia860 concatenated across all years.
Transforming raw EIA 860 DataFrames for generators_eia860 concatenated across all years.
Transforming raw EIA 860 DataFrames for plants_eia860 concatenated across all years.
Transforming raw EIA 860 DataFrames for boiler_generator_assn_eia860 concatenated across all years.
Transforming raw EIA 860 DataFrames for utilities_eia860 concatenated across all years.
Transforming raw EIA 923 DataFrames for generation_fuel_eia923 

In [18]:
# create an eia transformed dfs dictionary
eia_transformed_dfs = eia923_transformed_dfs.copy()
eia_transformed_dfs.update(eia923_transformed_dfs.copy())
# convert types..
eia_transformed_dfs = pudl.helpers.convert_dfs_dict_dtypes(
    eia_transformed_dfs, 'eia')

entities_dfs, eia_transformed_dfs = pudl.transform.eia.transform(
    eia_transformed_dfs,
    eia860_years=eia860_years,
    eia923_years=eia923_years,
)

Harvesting IDs & consistently static attributes for EIA plants


/Users/aesharpe/miniconda3/envs/pudl-dev/lib/python3.8/site-packages/pandas/core/missing.py:49: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x
/Users/aesharpe/miniconda3/envs/pudl-dev/lib/python3.8/site-packages/pandas/core/missing.py:49: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x


KeyError: "['balancing_authority_code_eia'] not in index"

In [9]:
list(eia_transformed_dfs.keys())

['generation_fuel_eia923',
 'boiler_fuel_eia923',
 'generation_eia923',
 'coalmine_eia923',
 'fuel_receipts_costs_eia923']

In [14]:
eia_transformed_dfs['generation_fuel_eia923'].sort_values('report_date', ascending=False)

,fuel_consumed_for_electricity_mmbtu,fuel_consumed_for_electricity_units,fuel_consumed_mmbtu,fuel_consumed_units,fuel_mmbtu_per_unit,fuel_type,fuel_type_code_aer,net_generation_mwh,nuclear_unit_id,plant_id_eia,prime_mover_code,fuel_type_code_pudl,report_date
13777,32092.0,0.0,32092.0,0.0,0.000,SUN,SUN,3525.000,<NA>,62810,PV,solar,2018-12-01
5698,0.0,0.0,0.0,0.0,0.000,PG,WOO,0.000,<NA>,50224,IC,gas,2018-12-01
5694,5791.0,0.0,5791.0,0.0,0.000,WAT,HYC,636.130,<NA>,50219,HY,hydro,2018-12-01
5695,493.0,0.0,493.0,0.0,0.000,WAT,HYC,54.132,<NA>,50223,HY,hydro,2018-12-01
5696,511.0,511.0,742.0,742.0,1.000,NG,NG,61.411,<NA>,50224,IC,gas,2018-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8476,0.0,0.0,0.0,0.0,0.000,DFO,DFO,0.000,<NA>,56234,CA,oil,2018-01-01
8475,544637.0,565563.0,544637.0,565563.0,0.963,NG,NG,62914.000,<NA>,56233,CT,gas,2018-01-01
8474,0.0,0.0,0.0,0.0,0.000,NG,NG,11517.000,<NA>,56233,CA,gas,2018-01-01
8473,104563.0,99869.0,104563.0,99869.0,1.047,NG,NG,10688.000,<NA>,56232,GT,gas,2018-01-01
